## Ejercicios de pair programming 11 enero : ETL 2.2

Tendréis que crear una clase con el código que usamos en los ejercicios de pair programming de ETL Transformación I y II.

In [1]:
import requests
import pandas as pd
import numpy as np

pd.options.display.max_columns = None

In [2]:
df = pd.read_csv("../datos/tiburon8.csv", index_col=0) #abrimos el archivo tiburon8 y lo guardamos en df

In [3]:
"""Definimos la clase Extracion_union que llama a la API de 7timer y devuelve el producto meteo de los países indicados y une los datos con     
    el dataframe de ataques de tiburones según los países indicados.
    
    Recibe dos atributos, un diccionario de paises y coordenadas de los países y el dataframe donde uniremos los datos de la api
    Métodos:
        llamado_API: llama a la API 7timer de los países indicados y devuelve un dataframe con los datos meteo de estos países
        filtrado: filtra el df original con los paíes que indiquemos en list_paises y devuelve del df filtrado
        limpiar_clima: hace las modificaciones necesarias de limpieza del df con los datos meteo y añade la columna de país. Devuelve el df limpio
        union_guardado: une el df de los ataques de tiburones y del clima limpio por la columna country y guarda el df en un csv con el nombre que indiquemos."""
class Extraccion_union:
    def __init__(self, dicc_paises, df):
        self.dicc_paises = dicc_paises
        self.df = df

    def llamada_API (self):
        df_meteo = pd.DataFrame()  #Creamos un df vacio para ir añadiendo los resultados de las peticiones de informacion a la API por cada pais
        for k , v in self.dicc_paises.items():   #iteramos por cada key y value de nuestro diccionario
            lon = v[0]   #la longitud sera el valor 0 de nuestra lista de values
            lat= v[1]     #la latitud sera el valor 1 de nuestra lista de values
            url = f'http://www.7timer.info/bin/api.pl?lon=-{lon}&lat={lat}&product=meteo&output=json'    #llamamos a la url de la API
        
            response = requests.get(url=url)
            response.status_code
            response.reason

            meteo = response.json()
            meteo_data = meteo["dataseries"]  #necesitamos la info de la columna dataseries 
            meteo_df = pd.json_normalize(meteo_data)   #generamos un df 
            meteo_df["country"] = k.lower()   #creamos una columna llamada pais 
            df_meteo = pd.concat([df_meteo , meteo_df], axis= 0 , join= 'outer')  #concatenamos cada resultado de cada pais al df vacio
        return df_meteo 

    def filtrado (self, list_paises):
        self.list_paises = list_paises

        df_filtrado = self.df[self.df['country'].isin(self.list_paises)] 
        return df_filtrado


    def limpiar_clima(self, data):
        self.data = data

        try:
            self.data['rh_profile']= self.data['rh_profile'].apply(ast.literal_eval)        
            self.data['wind_profile']= self.data['wind_profile'].apply(ast.literal_eval)

        except:
            rh = self.data['rh_profile'].apply(pd.Series)
            for i in range(len(rh.columns)): # Iteramos por las columnas de rh y definimos el nombre de la columna como rh_+valor de layer y los valores cada value de rh
                nombre = "rh_" + str(rh.iloc[0][i]['layer']) 
                valores = list(rh[i].apply(pd.Series)["rh"] )
                self.data.insert(i, nombre, valores) #Insertamos las columnas en el df clima

            wind = self.data['wind_profile'].apply(pd.Series)
            for i in range(len(wind.columns)): 
                nombre_dir = "wdir_" + str(wind.iloc[0][i]['layer']) 
                valores_dir = list(wind[i].apply(pd.Series)["direction"] )

                nombre_speed = "wspeed_" + str(wind.iloc[0][i]['layer']) 
                valores_speed = list(wind[i].apply(pd.Series)["speed"] )

                data.insert(i, nombre_dir, valores_dir) # Insertamos las columnas generadas en el df de clima
                data.insert(i, nombre_speed, valores_speed)

            data = self.data.groupby('country').mean().reset_index() #Agrupamos el df por la columna country y calculamos la media del resto 
        return data

    def union_guardado(self, data, df_filtrado, nombre_guardado):
        self.data = data
        self.df_filtrado = df_filtrado
        self.nombre_guardado = nombre_guardado

        df_final = self.df_filtrado.merge(self.data, how='left',  on='country') #Unimos los df

        df_final.to_csv(f'../datos/{self.nombre_guardado}') #Guardamos el df creado
        return df_final.head()

In [4]:
#Creamos el diccionario con los datos proporcionados
dicc_paises = {"USA" : [-100.445882, 39.7837304] , "Australia" : [134.755 , -24.7761086] , "South Africa" : [24.991639 , -28.8166236] , 
"New Zealand" : [172.8344077, -41.5000831], "Papua New Guinea": [144.2489081, -5.6816069]}  

In [5]:
#Llamamos a la clase y almacenamos en la variable api
api = Extraccion_union(dicc_paises, df)

In [6]:
df_meteo = api.llamada_API() #Aplicamos el método llamada_API y vemos el df 
df_meteo.head(2)

,timepoint,cloudcover,highcloud,midcloud,lowcloud,rh_profile,wind_profile,temp2m,lifted_index,rh2m,msl_pressure,prec_type,prec_amount,snow_depth,wind10m.direction,wind10m.speed,country
0,3,1,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 2}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 235, 'speed':...",13,15,3,1026,none,0,0,195,3,usa
1,6,2,-9999,-9999,-9999,"[{'layer': '950mb', 'rh': 5}, {'layer': '900mb...","[{'layer': '950mb', 'direction': 220, 'speed':...",14,10,9,1026,none,0,0,215,3,usa


In [7]:
#Definimos la lista países por la que queremos filtrar
lista_paises = ['usa', 'australia', 'new zealand', 'south africa', 'papua new guinea']

In [8]:
# Aplicamos el método de filtrado y vemos el df
df_filtrado = api.filtrado(lista_paises)
df_filtrado.head(2)

,case_number,year,type,country,activity,injury,mes,deceso,genero,especies,edad
0,2018.06.25,2018,Boating,usa,Paddling,"No injury to occupant, outrigger canoe and pad...",Jun,N,F,White,57.0
2,2018.05.26.b,2018,Unprovoked,usa,Walking,Lower left leg bitten,May,N,M,Bull,15.0


In [9]:
#Aplicamos el metodo limpiar_clima y vemos el df
meteo_limpio = api.limpiar_clima(df_meteo)
meteo_limpio.head(2)

/tmp/ipykernel_3599/125588642.py:65: FutureWarning: The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  data = self.data.groupby('country').mean().reset_index() #Agrupamos el df por la columna country y calculamos la media del resto


,country,wspeed_950mb,wspeed_900mb,wspeed_850mb,wspeed_800mb,wspeed_750mb,wspeed_700mb,wspeed_650mb,wspeed_600mb,wspeed_550mb,wspeed_500mb,wspeed_450mb,wspeed_400mb,wspeed_350mb,wspeed_300mb,wspeed_250mb,wspeed_200mb,wdir_200mb,wdir_250mb,wdir_300mb,wdir_350mb,wdir_400mb,wdir_450mb,wdir_500mb,wdir_550mb,wdir_600mb,wdir_650mb,wdir_700mb,wdir_750mb,wdir_800mb,wdir_850mb,wdir_900mb,wdir_950mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.speed
0,australia,3.3750,3.34375,3.203125,3.1875,3.078125,3.062500,3.125000,3.3125,3.500000,3.562500,3.578125,3.562500,3.593750,3.828125,4.265625,5.046875,137.343750,121.796875,121.875000,127.18750,125.00000,103.593750,101.71875,94.296875,101.484375,84.53125,81.171875,73.046875,65.781250,64.921875,67.265625,72.812500,13.421875,11.5000,8.109375,5.593750,4.140625,3.546875,3.171875,3.265625,3.046875,3.18750,3.562500,3.359375,2.53125,2.171875,4.265625,5.828125,97.5,3.015625,-9999.0,-9999.0,-9999.0,25.828125,-2.78125,10.359375,1016.171875,2.546875,0.0,3.171875
1,new zealand,3.9375,3.78125,3.703125,3.7500,3.640625,3.640625,3.703125,3.6875,3.734375,3.859375,3.968750,4.265625,5.015625,5.296875,6.062500,6.109375,231.484375,187.578125,196.328125,170.15625,162.96875,146.015625,145.46875,137.656250,132.421875,131.09375,126.250000,129.531250,126.328125,129.140625,113.906250,116.328125,13.125000,13.4375,6.218750,3.765625,3.421875,2.921875,2.062500,1.343750,0.937500,1.46875,2.453125,4.812500,5.40625,6.421875,6.953125,2.718750,97.5,7.187500,-9999.0,-9999.0,-9999.0,15.062500,10.18750,10.468750,1019.281250,2.187500,0.0,3.546875


In [10]:
#Aplicamos el mñetodo union_guardado y guardamos con el nombre tiburon10 y podemos ver el resultado 
api.union_guardado(meteo_limpio, df_filtrado, 'tiburon10.csv')

,case_number,year,type,country,activity,injury,mes,deceso,genero,especies,edad,wspeed_950mb,wspeed_900mb,wspeed_850mb,wspeed_800mb,wspeed_750mb,wspeed_700mb,wspeed_650mb,wspeed_600mb,wspeed_550mb,wspeed_500mb,wspeed_450mb,wspeed_400mb,wspeed_350mb,wspeed_300mb,wspeed_250mb,wspeed_200mb,wdir_200mb,wdir_250mb,wdir_300mb,wdir_350mb,wdir_400mb,wdir_450mb,wdir_500mb,wdir_550mb,wdir_600mb,wdir_650mb,wdir_700mb,wdir_750mb,wdir_800mb,wdir_850mb,wdir_900mb,wdir_950mb,rh_950mb,rh_900mb,rh_850mb,rh_800mb,rh_750mb,rh_700mb,rh_650mb,rh_600mb,rh_550mb,rh_500mb,rh_450mb,rh_400mb,rh_350mb,rh_300mb,rh_250mb,rh_200mb,timepoint,cloudcover,highcloud,midcloud,lowcloud,temp2m,lifted_index,rh2m,msl_pressure,prec_amount,snow_depth,wind10m.speed
0,2018.06.25,2018,Boating,usa,Paddling,"No injury to occupant, outrigger canoe and pad...",Jun,N,F,White,57.0,3.328125,3.734375,3.984375,4.156250,4.265625,4.500000,4.609375,4.96875,5.312500,6.00000,6.593750,7.015625,7.484375,8.218750,8.765625,9.187500,288.437500,281.640625,274.296875,289.140625,289.453125,293.828125,296.718750,286.640625,292.343750,282.96875,275.078125,283.828125,296.25000,281.718750,258.437500,244.062500,3.656250,3.937500,3.781250,3.109375,2.312500,2.171875,2.609375,2.953125,3.640625,4.187500,5.109375,4.390625,3.671875,4.656250,5.015625,4.187500,97.5,4.062500,-9999.0,-9999.0,-9999.0,12.203125,9.859375,3.671875,1013.640625,0.609375,0.0,2.953125
1,2018.05.26.b,2018,Unprovoked,usa,Walking,Lower left leg bitten,May,N,M,Bull,15.0,3.328125,3.734375,3.984375,4.156250,4.265625,4.500000,4.609375,4.96875,5.312500,6.00000,6.593750,7.015625,7.484375,8.218750,8.765625,9.187500,288.437500,281.640625,274.296875,289.140625,289.453125,293.828125,296.718750,286.640625,292.343750,282.96875,275.078125,283.828125,296.25000,281.718750,258.437500,244.062500,3.656250,3.937500,3.781250,3.109375,2.312500,2.171875,2.609375,2.953125,3.640625,4.187500,5.109375,4.390625,3.671875,4.656250,5.015625,4.187500,97.5,4.062500,-9999.0,-9999.0,-9999.0,12.203125,9.859375,3.671875,1013.640625,0.609375,0.0,2.953125
2,2018.05.24,2018,Provoked,australia,Feeding sharks,Minor bite to hand by captive shark. PROVOKED ...,May,N,M,Grey,32.0,3.375000,3.343750,3.203125,3.187500,3.078125,3.062500,3.125000,3.31250,3.500000,3.56250,3.578125,3.562500,3.593750,3.828125,4.265625,5.046875,137.343750,121.796875,121.875000,127.187500,125.000000,103.593750,101.718750,94.296875,101.484375,84.53125,81.171875,73.046875,65.78125,64.921875,67.265625,72.812500,13.421875,11.500000,8.109375,5.593750,4.140625,3.546875,3.171875,3.265625,3.046875,3.187500,3.562500,3.359375,2.531250,2.171875,4.265625,5.828125,97.5,3.015625,-9999.0,-9999.0,-9999.0,25.828125,-2.781250,10.359375,1016.171875,2.546875,0.0,3.171875
3,2018.04.25.a,2018,Unprovoked,australia,Surfing,"No injury, knocked off board by shark",Apr,N,M,Unspecified,60.0,3.375000,3.343750,3.203125,3.187500,3.078125,3.062500,3.125000,3.31250,3.500000,3.56250,3.578125,3.562500,3.593750,3.828125,4.265625,5.046875,137.343750,121.796875,121.875000,127.187500,125.000000,103.593750,101.718750,94.296875,101.484375,84.53125,81.171875,73.046875,65.78125,64.921875,67.265625,72.812500,13.421875,11.500000,8.109375,5.593750,4.140625,3.546875,3.171875,3.265625,3.046875,3.187500,3.562500,3.359375,2.531250,2.171875,4.265625,5.828125,97.5,3.015625,-9999.0,-9999.0,-9999.0,25.828125,-2.781250,10.359375,1016.171875,2.546875,0.0,3.171875
4,2018.04.22,2018,Unprovoked,south africa,Paddle-skiing,"No injury, shark bit hole in ski",Apr,N,M,White,33.0,2.578125,2.343750,2.234375,2.265625,2.234375,2.359375,2.406250,2.43750,2.609375,2.96875,3.453125,3.765625,3.875000,3.906250,4.156250,4.500000,214.296875,228.359375,229.609375,230.390625,228.281250,224.218750,222.890625,221.484375,217.109375,221.25000,219.609375,207.500000,180.62500,166.015625,138.203125,119.453125,13.140625,10.546875,8.281250,8.046875,9.359375,9.750000,10.140625,9.140625,6.218750,5.046875,2.656250,1.359375,2.437500,4.156250,5.625000,7.328125,97.5,5.390625,-9999.0,-9999.0,-9999.0